In [ ]:
######################################################################################
# Using a Policy Gradient Model to beat the CPU at the PLE pong environment
# Implementing the REINFORCE algorithm (Monte Carlo policy gradient)

# Need to: 
# - Implement policy gradient -> DONE
# - Add reward-to-go -> DONE
# - Add discounting -> DONE
# - Add baseline -> DONE (simple mean baseline)
# - Add a replay buffer

# Karphathy's took 30 hours of training & 10,000 episodes
# Karpathy used a larger layer 200, lr 1e-4, batch was done over 10 episodes

######################################################################################

In [ ]:
from ple.games.pong import Pong
from ple import PLE
import pygame
import numpy as np
import import_ipynb
import random
import pickle

from Policy_Gradient_Model import *
from Utils import *

##############################
# VARIABLES 
RENDER = True
MAX_SCORE = 5
NB_FRAMES = 10000
EPISODES = 10000
##############################

policy_model = agent()
reward_tally = []

for e in range(1000, EPISODES + 1):
    
    print("Starting Episode {}".format(e+1))
    
    random_num = random.randint(1, 100)
        
    pygame.init()
    game = Pong(width=512, height=400, MAX_SCORE=MAX_SCORE)
    game.screen = pygame.display.set_mode(game.getScreenDims(), 0, 32)
    env = PLE(game, fps=30, 
              display_screen=RENDER, 
              force_fps=False,
              reward_values={'tick': 0.01, 'win': 0.0, 'loss': 0.0, 'positive': 1.0, 'negative': -1.0},
             )
    env.init()
    
    # get the possible actions
    possible_actions = game.actions 
    action_names = ['up', 'down']
    
    state = process_state(env.getGameState())
    total_reward = 0
    rewards = []
    states = []
    actions = []
    
    # for some reason breaks without this    
    env.act(env.NOOP)
    reward = 0.0

    for f in range(NB_FRAMES):           
        
        action = policy_model.act(state) 
        reward = env.act(possible_actions[action_names[action]])
        next_state = process_state(env.getGameState())         
        
        rewards.append(reward)
        states.append(state)
        actions.append(action)
        
        state = next_state
        total_reward += reward
        
        #check if the game is over
        if env.game_over():            
            pygame.quit()
            
            policy_model.train(states, rewards, actions)  
            print("total reward after {} episodes is {}".format(e, total_reward))
            reward_tally.append(total_reward)
            
            if e == 1000:
                policy_model.load_model('model_pong.h5')                
        
            break      
            
    # Save the model  
    if e % 50 == 0:
        policy_model.save_model('model_pong.h5')

    # save the results
    if e % 200 == 0:        
        with open("./Rewards_Files/rewards-ep-" + str(e) + ".txt", "wb") as f:
            pickle.dump(reward_tally, f)
